# RL solutions for combinatorial games: Temporal Difference Methods

**This notebook presents the implementation with little to no explanation, all details are covered in [this notebook](nim_MC_on_policy.ipynb)**

In [78]:
import numpy as np
from tqdm import tqdm

In [79]:
np.random.seed(42)

In [80]:
# A Simple Nim Game
class SimpleNim:
    def __init__(self, state):
        self.heaps = state
        self.turn = 0
        self.num_heaps = len(self.heaps)

    def get_state(self):
        if isinstance(self.heaps, list):
            return tuple(self.heaps)
        else:
            return tuple(self.heaps.tolist())

    def get_num_heaps(self):
        return self.num_heaps

    def get_heap_size(self, heap_index):
        if 0 <= heap_index < self.num_heaps:
            return self.heaps[heap_index]
        return None

    def is_game_over(self):
        return all(pebbles == 0 for pebbles in self.heaps)

    def make_move(self, heap_index, num_pebbles):
        if self.is_game_over():
            return False

        # Validate the move
        if 0 <= heap_index < self.num_heaps and 0 < num_pebbles <= self.heaps[heap_index]:
            self.heaps[heap_index] -= num_pebbles
            self.turn = 1 - self.turn  # Switch turns
            return True
        return False

    def winner(self):
        if self.is_game_over():
            return 1-self.turn
        return None

In [81]:
def nim_sum(s):
    xor = 0
    for heap in s:
        xor ^= heap
    return xor

In [82]:
def play_optimal(game):
    if game.is_game_over():
        return
    heaps = game.get_state()

    nimber = nim_sum(heaps)
    largest_index, largest = max(enumerate(heaps), key=lambda x:x[1])

    if nimber != 0:
        game.make_move(largest_index, largest - (largest^nimber)) # optimal move
    else:
        game.make_move(largest_index, 1) # take the move that progresses the game to the least possible extent and hope for a blunder

In [83]:
# simple simulation of a nim game with both players using the optimal strategy
heaps = [10] * 3
game = SimpleNim(heaps)
while not game.is_game_over():
    play_optimal(game)
    print(game.get_state())
print("The winner is Player ", game.winner())

(0, 10, 10)
(0, 9, 10)
(0, 9, 9)
(0, 8, 9)
(0, 8, 8)
(0, 7, 8)
(0, 7, 7)
(0, 6, 7)
(0, 6, 6)
(0, 5, 6)
(0, 5, 5)
(0, 4, 5)
(0, 4, 4)
(0, 3, 4)
(0, 3, 3)
(0, 2, 3)
(0, 2, 2)
(0, 1, 2)
(0, 1, 1)
(0, 0, 1)
(0, 0, 0)
The winner is Player  0


## Defining State and Action Spaces:
**The State** is given by a configuration of heaps($\N_0$ means the set of naturals that includes zero):
$$ \mathcal S = \N_0^h $$
where h is the number of heaps
**Actions** are given by two tuples of heap number and number of pebbles to be taken out:
$$ \mathcal A(s) = H(s) \times \N $$ 
where $\N$ does not include zero and $H(s)$ is the set of heaps having non zero number of pebbles in the state s

In [84]:
# helper function that evaluates a policy based on whether the turn ends on a nim sum of 0
def is_optimal(s, a):
    s = list(s)
    if a[1] == 0: return False
    s[a[0]] -= a[1]
    return nim_sum(s) == 0

In [85]:
# for simplicity consider the version of the problem that starts off with same number of pebbles in each heap and 3 heaps
num_heaps = 3
initial_pebbles = 5
max_episodes = 100000

In [86]:
# Defining a class for epsilon greedy policy that has methods to update epsilon to account for GLIE conditions and to sample from epsilon greedy policy
class EpsilonGreedyPolicy:
    def __init__(self):
        self.epsilon = 1

    def update(self, epsilon):
        self.epsilon = epsilon

    def policy(self, s, Q):
        if s not in Q or np.random.random() < self.epsilon:
            index = np.random.randint(0, len(s))
            while s[index] == 0:
                index = np.random.randint(0, len(s))
            return (index, 1)
        else:
            return max(Q[s], key=Q[s].get)

    # Returns a policy that does not explore
    def final_policy(self, s, Q):
        if s not in Q or len(Q[s]) == 0:
            return (0,0)
        return max(Q[s], key=Q[s].get)

In [87]:
class SimpleGreedyPolicy:
    def update(self, epsilon): pass

    def policy(self, s, Q):
        if s not in Q:
            index = np.random.randint(0, len(s))
            while s[index] == 0:
                index = np.random.randint(0, len(s))
            return (index, 1)
        return max(Q[s], key=Q[s].get)

    def final_policy(self, s, Q):
        if s not in Q:
            return (0,0)
        return max(Q[s], key=Q[s].get)

In [88]:
def sample_step_zero_on_losing(game, A):
    game.make_move(A[0],A[1])
    play_optimal(game)
    S_next = game.get_state()
    if game.is_game_over() and game.winner() == 0: return 1, S_next
    else: return 0, S_next

### SARSA: On policy TD(0)

In [89]:
# SARSA with exploring starts
def sarsa(gamma=1, policy=EpsilonGreedyPolicy(), step_size=0.02, es=True, sample_step=sample_step_zero_on_losing):
    Q = {} # take default to be zero

    policy.update(0.05)

    for i in tqdm(range(max_episodes)):
        S = np.random.randint(0, initial_pebbles+1, num_heaps)
        while all(S == 0):
            S = np.random.randint(0, initial_pebbles+1, num_heaps)
        heap_num = np.random.randint(0,num_heaps)
        while S[heap_num] == 0:
            heap_num = np.random.randint(0,num_heaps)
        game = SimpleNim(S)

        S = tuple(S.tolist())
        if es: A = (heap_num, np.random.randint(1, S[heap_num]+1)) # Exploring start
        else: A = policy.policy(S, Q)

        while not game.is_game_over():
            R, S1 = sample_step(game, A)
            if game.is_game_over():
                A1 = (0,0)
            else:
                A1 = policy.policy(S1, Q)

            if S1 not in Q:
                Q[S1] = {}
            if A1 not in Q[S1]:
                Q[S1][A1] = 0
            if S not in Q:
                Q[S] = {}
            if A not in Q[S]:
                Q[S][A] = 0
            target = R + gamma*Q[S1][A1]
            delta = target-Q[S][A]
            Q[S][A] = Q[S][A] + step_size*delta

            A = A1
            S = S1

    return policy, Q

In [90]:
# Epsilon Greedy and exploring starts
policy, Q = sarsa()

print(Q)

100%|██████████| 100000/100000 [00:02<00:00, 40400.12it/s]

{(2, 0, 2): {(2, 1): 0.010172005062978171, (0, 1): 0.006680444109593852, (2, 2): 0.0, (0, 2): 0.0}, (3, 4, 2): {(0, 1): 0.00927048048344161, (2, 2): 0.015009217947385984, (0, 3): 0.019508952893018613, (1, 3): 0.24058532023446186, (0, 2): 0.3065759333657823, (1, 1): 0.24031462903734857, (1, 4): 0.007832213273179445, (1, 2): 0.01722731828429149, (2, 1): 0.39867617387195275}, (1, 0, 1): {(2, 1): 0.014135328443806194, (0, 1): 0.00010372108720674622}, (0, 0, 0): {(0, 0): 0}, (0, 2, 2): {(1, 1): 0.016559484004412456, (2, 2): 0.0, (2, 1): 0.004878302975632682, (1, 2): 0.0}, (2, 2, 4): {(2, 2): 0.011772073058729808, (1, 1): 0.3527179171752397, (0, 2): 0.016182369091054714, (2, 4): 0.3077928672500807, (2, 3): 0.23353689309886658, (0, 1): 0.3244365439014648, (1, 2): 0.010839928441612839, (2, 1): 0.008934603089724375}, (0, 1, 1): {(1, 1): 0.00817020736480991, (2, 1): 0.00826572822382068}, (2, 1, 3): {(1, 1): 0.01590383185144817, (0, 2): 0.03781142814825146, (2, 2): 0.02511684686222652, (2, 3): 0.

In [91]:
import itertools

def evaluate_on_winning_positions(policy, Q):
    fumbles = 0
    num_winning_pos = 0
    for s in itertools.product(range(initial_pebbles), repeat=num_heaps):
        if s == (0)*num_heaps: continue
        action = policy.final_policy(s, Q)
        if nim_sum(s) == 0: continue
        if not is_optimal(s, action):
            fumbles+=1
        num_winning_pos+=1
    return fumbles, num_winning_pos

In [92]:
def evaluate_using_fumbles(policy, Q):
    fumbles, num_winning_pos = evaluate_on_winning_positions(policy, Q)
    print('number of fumbles:', fumbles)
    print('total number of winning positions:', num_winning_pos)
    print('percentage of winning positions fumbled: {:.2f} %'.format(fumbles/num_winning_pos*100))

In [93]:
evaluate_using_fumbles(policy, Q)

number of fumbles: 31
total number of winning positions: 106
percentage of winning positions fumbled: 29.25 %


In [94]:
# Epsilon Greedy but no exploring starts
policy, Q = sarsa(es=False)

print(Q)

100%|██████████| 100000/100000 [00:02<00:00, 39533.15it/s]

{(0, 3, 3): {(1, 1): 0.037824404674171246, (2, 1): 0.02279573680112294}, (5, 3, 4): {(2, 1): 0.032889260821572425, (0, 1): 0.22305672113661681, (1, 1): 0.02640301088450344}, (0, 2, 2): {(2, 1): 0.0179681704635771, (1, 1): 0.035087939210798266}, (0, 1, 1): {(2, 1): 0.023791689145528913, (1, 1): 0.0005674356323956036}, (0, 0, 0): {(0, 0): 0}, (1, 0, 2): {(0, 1): 0.6094228242112895, (2, 1): 0.44485132244781495}, (2, 0, 3): {(2, 1): 0.5366832371100378, (0, 1): 0.03379270002665906}, (1, 2, 3): {(1, 1): 0.041233031174576384, (2, 1): 0.4806583232284759, (0, 1): 0.033414606951486604}, (4, 2, 4): {(2, 1): 0.47436434935686483, (0, 1): 0.18153909280232938, (1, 1): 0.1777715808455672}, (1, 1, 0): {(0, 1): 0.0013508840528486169, (1, 1): 0.05437806256021763}, (2, 3, 0): {(1, 1): 0.675344746312926, (0, 1): 0.0235864361253679}, (4, 3, 0): {(0, 1): 0.6537626006911748, (1, 1): 0.009016543935712731}, (1, 2, 0): {(0, 1): 0.703323204073772, (1, 1): 0.5371152688761612}, (3, 0, 3): {(2, 1): 0.039075467356460

In [95]:
evaluate_using_fumbles(policy, Q)

number of fumbles: 70
total number of winning positions: 106
percentage of winning positions fumbled: 66.04 %


In [96]:
# Apply the new rewards model with negative rewards for losses
def sample_step_negative_on_losing(game, A):
    game.make_move(A[0],A[1])
    play_optimal(game)
    S_next = game.get_state()
    if game.is_game_over() and game.winner() == 0: return 1, S_next
    elif game.is_game_over(): return -1, S_next
    else: return 0, S_next

In [97]:
# Epsilon Greedy with exploring starts negative rewards on losing
policy, Q = sarsa(es=False, sample_step=sample_step_negative_on_losing)

print(Q)

100%|██████████| 100000/100000 [00:02<00:00, 36472.46it/s]

{(0, 0, 0): {(0, 0): 0}, (1, 1, 0): {(1, 1): -0.9054658922242375, (0, 1): -0.9833169409674823}, (0, 2, 2): {(2, 1): -0.907029720059136, (1, 1): -0.9365356328425813}, (4, 3, 2): {(1, 1): -0.6613603157604023, (0, 1): 0.13161353966923678, (2, 1): 0.010227608517183776}, (0, 1, 1): {(1, 1): -0.8640108391123644, (2, 1): -0.9506393925211725}, (2, 2, 1): {(2, 1): 0.14213878923507572, (1, 1): -0.8874100071461809, (0, 1): -0.8809441262018514}, (2, 2, 2): {(2, 1): 0.021996973263029688, (1, 1): 0.156300165566291, (0, 1): 0.01691670886569237}, (1, 2, 0): {(0, 1): 0.04397409388793316, (1, 1): 0.1561968080748286}, (1, 0, 1): {(0, 1): -0.9528962809825617, (2, 1): -0.9662320981420967}, (3, 1, 1): {(1, 1): -0.8722684465380847, (2, 1): -0.8702814194123946, (0, 1): -0.8713082630102866}, (4, 1, 0): {(1, 1): -0.9149682270339905, (0, 1): -0.9004735594921693}, (4, 4, 0): {(0, 1): -0.9062137647622877, (1, 1): -0.913909283316653}, (5, 5, 0): {(0, 1): -0.8794530369288533, (1, 1): -0.8792213119477805}, (3, 3, 0):

In [98]:
evaluate_using_fumbles(policy, Q)

number of fumbles: 70
total number of winning positions: 106
percentage of winning positions fumbled: 66.04 %


In [99]:
# Epsilon Greedy with exploring starts negative rewards on losing
policy, Q = sarsa(sample_step=sample_step_negative_on_losing)

print(Q)

100%|██████████| 100000/100000 [00:02<00:00, 40295.61it/s]

{(3, 0, 3): {(0, 1): -0.980825180309808, (2, 1): -0.9801141357232788, (2, 2): -0.980418799803765, (0, 2): -0.9707399657134845, (2, 3): -0.9867449857735072, (0, 3): -0.985629367099475}, (5, 1, 3): {(1, 1): -0.8932789472984958, (2, 3): -0.6478769905343843, (0, 2): -0.039841321894281155, (0, 4): -0.42294618009755974, (0, 1): -0.0271210362294313, (0, 5): -0.41329406254361906, (2, 2): -0.4737687491599865, (2, 1): -0.030965375919389984, (0, 3): -0.016493556917347267}, (2, 0, 2): {(0, 1): -0.9675609290809355, (2, 2): -0.9966445301160628, (0, 2): -0.9971425189397003, (2, 1): -0.9890205324936565}, (1, 0, 1): {(2, 1): -0.9865700984542729, (0, 1): -0.9999395299640117}, (0, 0, 0): {(0, 0): 0}, (2, 2, 0): {(1, 2): -0.9968418585889973, (0, 1): -0.987027883767862, (1, 1): -0.9761835026984609, (0, 2): -0.9984739446094733}, (2, 2, 1): {(0, 1): -0.9483166989092305, (2, 1): -0.01847397152814102, (1, 1): -0.9539354257785827, (1, 2): -0.8096426805734307, (0, 2): -0.7513202709314701}, (3, 3, 1): {(0, 1): -0

In [100]:
evaluate_using_fumbles(policy, Q)

number of fumbles: 14
total number of winning positions: 106
percentage of winning positions fumbled: 13.21 %


In [101]:
# providing a way to check performance more reliably
def evaluate_using_fumbles_averaged(train, num_iters=5):
    num_winning_pos = 0
    total_fumbles = 0
    for i in range(num_iters):
        print(f'iteration {i+1}')
        policy, Q = train(sample_step=sample_step_negative_on_losing)

        fumbles, num_winning_pos = evaluate_on_winning_positions(policy, Q)
        total_fumbles += fumbles
    return total_fumbles, num_winning_pos

In [ ]:
# num_iters = 5
# total_fumbles, num_winning_pos = evaluate_using_fumbles_averaged(sarsa, num_iters=num_iters)
# print('average number of fumbles: ', total_fumbles/num_iters)
# print('fumble rate: {:.2f}%'.format(total_fumbles/float(num_winning_pos*num_iters)*100))

iteration 1


100%|██████████| 100000/100000 [00:02<00:00, 40929.79it/s]


iteration 2


100%|██████████| 100000/100000 [00:02<00:00, 39122.59it/s]


iteration 3


...

iteration 4


100%|██████████| 100000/100000 [00:02<00:00, 39803.06it/s]


iteration 5


100%|██████████| 100000/100000 [00:02<00:00, 40319.30it/s]

average number of fumbles:  21.4
fumble rate: 20.19%


### Q-Learning: Off-policy TD(0)

In [125]:
# Q-Learning
def qlearning(gamma=1, policy=EpsilonGreedyPolicy(), step_size=0.02, es=True, sample_step=sample_step_zero_on_losing):
    Q = {} # take default to be zero

    for i in tqdm(range(max_episodes)):
        S = np.random.randint(0, initial_pebbles+1, num_heaps)
        while all(S == 0):
            S = np.random.randint(0, initial_pebbles+1, num_heaps)
        heap_num = np.random.randint(0,num_heaps)
        while S[heap_num] == 0:
            heap_num = np.random.randint(0,num_heaps)
        game = SimpleNim(S)

        S = tuple(S.tolist())
        if es: A = (heap_num, np.random.randint(1, S[heap_num]+1)) # Exploring start
        else: A = policy.policy(S, Q)

        while not game.is_game_over():
            R, S1 = sample_step(game, A)
            if game.is_game_over():
                A1 = (0,0)
            else:
                A1 = policy.policy(S1, Q)

            if S1 not in Q:
                Q[S1] = {}

            if len(Q[S1]) != 0: Q_max = Q[S1][max(Q[S1])]
            else: Q_max = 0

            if S not in Q:
                Q[S] = {}
            if A not in Q[S]:
                Q[S][A] = 0
            target = R + gamma*Q_max
            delta = target-Q[S][A]
            Q[S][A] = Q[S][A] + step_size*delta

            A = A1
            S = game.get_state()

        if i >= 2: policy.update(2/(i+1))

    return policy, Q

In [126]:
policy, Q = qlearning(sample_step=sample_step_negative_on_losing)

print(Q)

100%|██████████| 100000/100000 [00:02<00:00, 42211.39it/s]

{(2, 2, 0): {(1, 1): -0.7641695381894223, (0, 1): -0.8794611310990302, (0, 2): -0.9995324004812008, (1, 2): -0.9995638990926906}, (2, 3, 2): {(2, 2): -0.8149040071045853, (0, 1): -0.46677166760966127, (1, 1): -0.6039160554594316, (0, 2): -0.7612331650300661, (2, 1): -0.44735968240552815, (1, 3): -0.3648089669776756, (1, 2): -0.4021295299922284}, (1, 1, 0): {(1, 1): -0.8032451915382393, (0, 1): -0.9999908866633044}, (0, 0, 0): {}, (2, 3, 1): {(1, 1): -0.855014663825985, (0, 1): -0.856956140309473, (2, 1): -0.9413445834658482, (1, 2): -0.8576479689773507, (1, 3): -0.8556695577730917, (0, 2): -0.8610161312111574}, (3, 3, 5): {(0, 1): -0.4676307673578351, (2, 2): -0.5406014452308734, (0, 3): -0.5181971966374148, (1, 2): -0.3933091179251836, (1, 1): -0.4738373978576282, (1, 3): -0.5400473205716425, (2, 5): -0.3077369211582889, (2, 3): -0.2081202886941515, (2, 4): -0.317089486033146, (0, 2): -0.3999983579190529, (2, 1): -0.33890694428171214}, (2, 2, 1): {(2, 1): -0.8685363338298566, (0, 2): 

In [127]:
evaluate_using_fumbles(policy, Q)

number of fumbles: 22
total number of winning positions: 106
percentage of winning positions fumbled: 20.75 %
